In [1]:
from conv_models import DeepSpeakerModel
from constants import NUM_FBANKS, NUM_FRAMES

In [2]:
model = DeepSpeakerModel(batch_input_shape=(None, None, NUM_FBANKS, 1))
# Load the checkpoint.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
model.m.load_weights('ResCNN_triplet_training_checkpoint_265.h5', by_name=True)

In [4]:
import tensorflow as tf
session = tf.keras.backend.get_session()

In [5]:
saver = tf.train.Saver()
saver.save(session, "out/vggvox.ckpt")

'out/vggvox.ckpt'

In [6]:
from constants import SAMPLE_RATE, NUM_FRAMES
SAMPLE_RATE, NUM_FRAMES

(16000, 160)

In [7]:
from glob import glob

wavs = glob('*.wav')
wavs

['khalil-nooh.wav',
 'husein-zolkepli.wav',
 'mas-aisyah.wav',
 'shafiqah-idayu.wav']

In [8]:
from audio import read_mfcc
import numpy as np

read_mfcc(wavs[0], SAMPLE_RATE).shape

/home/ubuntu/.local/lib/python3.6/site-packages/numba/core/errors.py:149: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


(183, 64)

In [9]:
read_mfcc(wavs[1], SAMPLE_RATE).shape

(335, 64)

In [10]:
def f(file):
    return np.array([np.expand_dims(read_mfcc(file, SAMPLE_RATE), -1)])

In [11]:
mfccs = [f(file) for file in wavs]

In [12]:
def pred(mfcc):
    return model.m.predict(mfcc)

In [13]:
r = [pred(mfcc) for mfcc in mfccs]

In [14]:
r = np.concatenate(r)
r.shape

(4, 512)

In [15]:
from scipy.spatial.distance import cdist

1 - cdist(r, r, metric='cosine')

array([[1.        , 0.33842053, 0.23876477, 0.23889499],
       [0.33842053, 1.        , 0.19461671, 0.24097245],
       [0.23876477, 0.19461671, 1.        , 0.321913  ],
       [0.23889499, 0.24097245, 0.321913  , 1.        ]])